# Dataset final
El objetivo de este notebook es crear el dataset de **productos.csv** el cual contendrá una categorización de los productos/negocios en cada una de las colonias prioritarias. Recordemos que para este punto, **microempresas_clean.csv** ya contiene los negocios de la CDMX que solicitaron apoyo financiero únicamente en las colonias prioritarias. 

In [38]:
import pandas as pd
data_path = '../../data/'

In [39]:
alcaldias_df = pd.read_csv(data_path + 'alcaldias_eci.csv')
negocios_df = pd.read_csv(data_path + 'microempresas_clean.csv')
colonias_df = pd.read_csv(data_path + 'colonias_prioritarias_clean.csv')

In [40]:
alcaldias_df.head()

,alcaldia,eci
0,cuauhtemoc,3.471986
1,venustiano carranza,2.501312
2,azcapotzalco,3.865509
3,coyoacan,2.800526
4,cuajimalpa de morelos,3.854617


In [41]:
negocios_df.head()

,ocupacion,alcaldia,c_p,colonia,giro,num_empleados
0,tiendas de ropa,alvaro obregon,1260,calzada jalalpa,tiendas de ropa,3
1,tiendas de abarrotes,alvaro obregon,1780,olivar de los padres,tiendas de abarrotes,0
2,tiendas de abarrotes,alvaro obregon,1780,olivar de los padres,tiendas de abarrotes,3
3,tiendas de abarrotes,alvaro obregon,1780,olivar de los padres,tiendas de abarrotes,0
4,venta de zapatos y productos de belleza,alvaro obregon,1290,presidentes,venta de zapatos y productos de belleza,0


In [42]:
colonias_df.head()

,alcaldia,colonia,casos_activos,poblacion_total,porcentaje_activos,cp
0,alvaro obregon,barrio norte,57,14367.414210,0.2562,1410
1,alvaro obregon,jalalpa el grande,47,10972.245610,0.2112,1377
2,alvaro obregon,jardines del pedregal,39,13817.000000,0.1753,1900
3,alvaro obregon,merced gomez,48,6992.834615,0.2157,1600
4,alvaro obregon,olivar del conde 2da seccion ii,37,8191.441839,0.1663,1408


## Tomando los negocios en las alcaldias con top 3 ECI

In [43]:
top = 3
best_alcaldias_eci_df = alcaldias_df.sort_values(by='eci')[::-1].head(top)
best_alcaldias = list(best_alcaldias_eci_df.alcaldia)
negocios_mh_df = negocios_df[negocios_df.alcaldia.isin(best_alcaldias)]
print(best_alcaldias)

['miguel hidalgo', 'azcapotzalco', 'cuajimalpa de morelos']


In [44]:
print(len(list(negocios_mh_df.colonia.unique())))
print(len(list(negocios_mh_df.ocupacion)))

13
353


In [45]:
categoria_productos_df = pd.read_csv(data_path + 'cat.csv')
categoria_productos_df.categoria = categoria_productos_df.categoria.str.lower()
categoria_productos_df.head()

,categoria,negocio
0,comida,tiendasabarrotes
1,comida,verduras
2,comida,cremeria
3,comida,casa_expendiopan
4,comida,abarrotes


In [46]:
negocios_ = list(categoria_productos_df.iloc[:, 1])
categorias_ = list(categoria_productos_df.iloc[:, 0])
cat_prod_dict = dict(zip(negocios_, categorias_))

In [47]:
stopwords_list = ['venta', 'de', 'el', 'la', 'los', 'las', 'uno', 'unos', 'un', 'ella', 'con', 'y', 'comerciante', 'comercio', 'vendedor', 'vendedora', 'producto', 'productos', 'servicio', 'servicios', 'para', 'sin', 'por', 'panadero', 'panadera', 'en', 'ante', 'bajo', 'cabe', 'contra', 'desde', 'hasta', 'hacia', 'dentro de', 'tras', 'sobre', 'estudiante', 'tecnico', 'tecnica', 'compra', 'dueo', 'duea', 'nombre', 'empleada', 'empleado', 'auto', 'ama', 'amo', 'negocio', 'propietario', 'propietaria', 'independiente']
stopwords = set()
for s in stopwords_list:
    stopwords.add(s)

def remove_stop_words(s):
    new_word = ''
    curr = s.lower().split(' ')
    for c in curr:
        if c not in stopwords:
            new_word += c
    return new_word

In [48]:
remove_stop_words('venta de zapatos y productos de belleza')

'zapatosbelleza'

In [49]:
categorias = []
negocios = []
not_seen = []
negocio = ''
for _, row in negocios_mh_df.iterrows():
    ocupacion = remove_stop_words(row.ocupacion)
    giro = remove_stop_words(row.giro)
    if len(ocupacion) + len(giro) > 0:
        if ocupacion == giro or len(ocupacion) == 0:
            negocio = giro
        elif len(giro) == 0:
            negocio = ocupacion
        else:
            negocio = ocupacion + '_' + giro 
        if negocio in cat_prod_dict:
            negocios.append(negocio)
            categorias.append(cat_prod_dict[negocio])
            continue
    not_seen.append((row.ocupacion,row.giro))

In [50]:
for ns in not_seen:
    print(ns)

('comerciante', 'comerciante')
('comerciante', 'comerciante')
('comerciante', 'comercio')
('comercio', 'comercio')
('comerciante', 'puesto')
('comerciante', 'comerciante')
('comerciante', 'comerciante')
('comerciante', 'comerciante')
('comerciante', 'comerciante')
('comerciante', 'comerciante')
('comerciante', 'comerciante')
('comerciante', 'comerciante')
('vendedora', 'comerciante')
('comerciante', 'comercio   servicio')
('comerciante', 'comercio   servicio')
('sin nombre', 'servicios')
('sin nombre', 'comercio')
('sin nombre', 'comercio')
('sin nombre', 'comercio')
('comerciante independiente', 'comercio   servicio')
('comerciante', 'comercio')


In [51]:
indexes_to_drop = set()
for ocupacion_to_drop, giro_to_drop in not_seen:
    idxs = list(
        negocios_mh_df.loc[(negocios_mh_df.ocupacion == ocupacion_to_drop) & (negocios_mh_df.giro == giro_to_drop)].index
    )
    for i in idxs: indexes_to_drop.add(i)

In [52]:
negocios_mh_df = negocios_mh_df.drop(indexes_to_drop)

In [53]:
print(len(categorias), len(negocios))

332 332


In [54]:
complexity_indices = []
for _, row in negocios_mh_df.iterrows():
    alcaldia = row.alcaldia
    eci = alcaldias_df[alcaldias_df.alcaldia==alcaldia].eci.values[0]
    complexity_indices.append(eci)

In [55]:
negocios_mh_df.drop(columns=["ocupacion", "giro"], inplace=True)
negocios_mh_df.insert(1, "eci", complexity_indices)
negocios_mh_df.insert(4, "negocio", negocios)
negocios_mh_df.insert(5, "categoria", categorias)

In [56]:
activos = []
for _, row in negocios_mh_df.iterrows():
    cp = row.c_p
    activos.append(colonias_df[colonias_df.cp == cp].porcentaje_activos.values[0])
negocios_mh_df.insert(6, "porcentaje_infectados", activos)

In [57]:
negocios_mh_df.head()

,alcaldia,eci,c_p,colonia,negocio,categoria,porcentaje_infectados,num_empleados
67,azcapotzalco,3.865509,13530,jaime torres bodet,zapatosbelleza,arreglosgeneral,0.1978,2
68,azcapotzalco,3.865509,2100,el rosario,manufacturados,industria,0.1573,0
69,azcapotzalco,3.865509,2459,tezozomoc,antojerias,restaurantes,0.1303,3
70,azcapotzalco,3.865509,6720,doctores,fotografia,fotografia,0.1169,3
71,azcapotzalco,3.865509,2459,tezozomoc,catalogo,ventaproductos,0.1303,2


In [58]:
negocios_mh_df.tail()

,alcaldia,eci,c_p,colonia,negocio,categoria,porcentaje_infectados,num_empleados
10373,miguel hidalgo,4.152792,11400,popotla,locatariopublico_abarrotes,comida,0.1528,1
10436,miguel hidalgo,4.152792,16090,xaltocan,costurera_reparacionesbordadosropamedida,ropa,0.2921,2
10451,miguel hidalgo,4.152792,11370,tlaxpana,chilessecos,comida,0.1573,1
10452,miguel hidalgo,4.152792,11370,tlaxpana,legumbres,comida,0.1573,1
10465,miguel hidalgo,4.152792,16450,santa maria nativitas,regalosnovedades,ventaproductos,0.2427,2


In [59]:
negocios_mh_df.to_csv(data_path+'datasetfinal.csv', index=False)